Breaking changes. Partitions columns are written do parquet files.

In [0]:
from pyspark.sql import functions as F

target_path = "abfss://unity@westus2uc.dfs.core.windows.net/partitions/table1"

data = [
    ("2025-12-28", "A", 100.0),
    ("2025-12-28", "B", 250.0),
    ("2025-12-29", "A", 75.5),
    ("2025-12-30", "C", 10.0),
]

df = (
    spark.createDataFrame(data, ["event_date", "category", "amount"])
         .withColumn("event_date", F.to_date("event_date"))
)

df_part = (
    df.withColumn("year",  F.year("event_date"))
      .withColumn("month", F.month("event_date"))
      .withColumn("day",   F.dayofmonth("event_date"))
)


(
    df_part.write
          .format("delta")
          .mode("overwrite") 
          .option("overwriteSchema", "true")
          .partitionBy("year", "month", "day")
          .save(target_path)
)


In [0]:
display(dbutils.fs.ls(target_path))